In [14]:
import pandas as pd
import sqlite3
import zipfile

In [4]:
! ls

CONTRIBUTING.md
LICENSE.md
README.md
Yue's_copy.ipynb
awesome.gif
movie_data_erd.jpeg
student.ipynb
zippedData


In [15]:
with zipfile.ZipFile('./zippedData/im.db.zip') as zipObj:
    zipObj.extractall(path='./zippedData/')
    

In [151]:
con = sqlite3.connect('zippedData/im.db')
cursor = con.cursor()


In [152]:
schema_df = pd.read_sql("""
                        SELECT *
                        FROM sqlite_master """, con)
schema_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [153]:
df_rating = pd.read_sql("""
                        SELECT *
                        FROM movie_ratings """, con)
df_rating.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [154]:
df_basics = pd.read_sql("""
                        SELECT *
                        FROM movie_basics """, con)
df_basics.head(50)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy
6,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller"
7,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy"
8,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History"
9,tt0144449,Nema aviona za Zagreb,Nema aviona za Zagreb,2012,82.0,Biography


In [158]:
df_basics.loc[df_basics['movie_id']=='tt1042974']

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
4721,tt1042974,Just Inès,Just Inès,2010,90.0,Drama


In [132]:
movie_title_rating = pd.read_sql("""
                                    SELECT 
                                        movie_ratings.movie_id, movie_ratings.averagerating, movie_akas.title
                                    FROM 
                                        movie_ratings
                                        LEFT JOIN movie_akas
                                            ON movie_ratings.movie_id = movie_akas.movie_id
                                    
                                    GROUP BY
                                        movie_ratings.movie_id""", con)
movie_title_rating.sort_values(by='averagerating', ascending=False).head(50)

,movie_id,averagerating,title
51109,tt5089804,10.0,Fly High: Story of the Disc Dog
65944,tt7259300,10.0,Calamity Kevin
71577,tt8730716,10.0,None
73616,tt9715646,10.0,None
65755,tt7227500,10.0,Ellis Island: The Making of a Master Race in A...
878,tt10378660,10.0,None
64646,tt6991826,10.0,A Dedicated Life: Phoebe Brand and the Legacy ...
9745,tt1770682,10.0,Freeing Bernie Baran
702,tt10176328,10.0,Exteriores: Mulheres Brasileiras na Diplomacia
49925,tt4944240,10.0,Dog Days in the Heartland


In [155]:
movie_titles_rating = pd.read_sql("""
                                    SELECT 
                                        movie_ratings.movie_id, movie_ratings.averagerating, movie_basics.primary_title AS movie
                                    FROM 
                                        movie_ratings
                                        LEFT JOIN movie_basics
                                            ON movie_ratings.movie_id = movie_basics.movie_id
                                    
                                    """, con)
movie_titles_rating.head(50)

,movie_id,averagerating,movie
0,tt10356526,8.3,Laiye Je Yaarian
1,tt10384606,8.9,Borderless
2,tt1042974,6.4,Just Inès
3,tt1043726,4.2,The Legend of Hercules
4,tt1060240,6.5,Até Onde?
5,tt1069246,6.2,Habana Eva
6,tt1094666,7.0,The Hammer
7,tt1130982,6.4,The Night Clerk
8,tt1156528,7.2,Silent Sonata
9,tt1161457,4.2,Vanquisher


In [157]:
movie_titles_rating.loc[movie_titles_rating['movie_id']=='tt1042974']

,movie_id,averagerating,movie
2,tt1042974,6.4,Just Inès


In [47]:
df_bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df_bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [24]:
df_rt_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t')
df_rt_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [27]:
df_rt_review = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep='\t', encoding='unicode escape')
df_rt_review.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [ ]:
df_rt

In [28]:
df_tmdb = pd.read_csv('zippedData/tmdb.movies.csv.gz')
df_tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [29]:
df_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
df_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [38]:
df_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.1+ KB


In [73]:
# df_budgets.sort_values(by='production_budget')

In [74]:
# df_budgets['production_budget'][0][0]

In [61]:
df_budgets['budgets'] = df_budgets['production_budget'].str.replace('$','').str.replace(',', '').astype(int)
df_budgets['dom_gross'] = df_budgets['domestic_gross'].str.replace('$','').str.replace(',', '').astype(int)


In [64]:
df_budgets['world_gross'] = df_budgets['worldwide_gross'].str.replace('$','').str.replace(',', '').astype('int64')

In [72]:
df_budgets.sort_values(by='budgets')

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,budgets,dom_gross,world_gross
5781,82,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041",1100,181041,181041
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0,1400,0,0
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338",5000,1338,1338
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495",6000,48482,240495
5776,77,"Dec 31, 2004",The Mongol King,"$7,000",$900,$900,7000,900,900
5775,76,"May 26, 2006",Cavite,"$7,000","$70,071","$71,644",7000,70071,71644
5774,75,"Oct 8, 2004",Primer,"$7,000","$424,760","$841,926",7000,424760,841926
5773,74,"Feb 26, 1993",El Mariachi,"$7,000","$2,040,920","$2,041,928",7000,2040920,2041928
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0,7000,0,0
5772,73,"Jan 13, 2012",Newlyweds,"$9,000","$4,584","$4,584",9000,4584,4584


In [159]:
budget_rating_df = pd.merge(movie_titles_rating,df_budgets, on='movie')
budget_rating_df.head(50)

,movie_id,averagerating,movie,id,release_date,production_budget,domestic_gross,worldwide_gross,budgets,dom_gross,world_gross
0,tt1043726,4.2,The Legend of Hercules,42,"Jan 10, 2014","$70,000,000","$18,848,538","$58,953,319",70000000,18848538,58953319
1,tt1094666,7.0,The Hammer,50,"Mar 21, 2008","$850,000","$442,638","$442,638",850000,442638,442638
2,tt3096900,6.5,The Hammer,50,"Mar 21, 2008","$850,000","$442,638","$442,638",850000,442638,442638
3,tt1171222,5.1,Baggage Claim,38,"Sep 27, 2013","$8,500,000","$21,569,509","$22,885,836",8500000,21569509,22885836
4,tt1210166,7.6,Moneyball,15,"Sep 23, 2011","$50,000,000","$75,605,492","$111,300,835",50000000,75605492,111300835
5,tt1212419,6.5,Hereafter,61,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270",50000000,32746941,108660270
6,tt1232829,7.2,21 Jump Street,44,"Mar 16, 2012","$42,000,000","$138,447,667","$202,812,429",42000000,138447667,202812429
7,tt1243974,5.4,Aloha,3,"May 29, 2015","$37,000,000","$21,052,030","$25,387,091",37000000,21052030,25387091
8,tt1261945,4.4,Sex and the City 2,16,"May 27, 2010","$95,000,000","$95,347,692","$294,680,778",95000000,95347692,294680778
9,tt1270761,5.6,Don't Be Afraid of the Dark,43,"Aug 26, 2011","$25,000,000","$24,046,682","$39,126,427",25000000,24046682,39126427
